In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import calendar
from datetime import datetime
from fpdf import FPDF

import matplotlib.pyplot as plt
from matplotlib import rcParams

In [3]:
class report_generator:
    
    def __init__(self, name):
        self.name = name + "_dir"
    
   
    def create_diagrams_folder(self):
    # Delete folder if exists and create it again
        try:
            shutil.rmtree(self.name)
            os.mkdir(self.name)
        except FileNotFoundError:
            os.mkdir(self.name)
            
    def plot(self, data: pd.DataFrame, filename: str, data_name: str) -> None:
        
        plt.figure(figsize=(12, 4))
        plt.grid(color='#F2F2F2', alpha=1, zorder=0)
        plt.hist(data[data_name], bins = 20, color='#087E8B', lw=3, zorder=5)
        plt.title(f'{data_name}', fontsize=17)
        plt.xlabel(data_name, fontsize=13)
        plt.xticks(fontsize=9)
        plt.ylabel('Frequence', fontsize=13)
        plt.yticks(fontsize=9)
        plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
        plt.close()
        return
    
    def add_diagrams_to_the_folder(self, diagrams):
        
        for diagram_name in diagrams.keys():
        # Save visualization
            self.plot(data=diagrams[diagram_name], filename=f'{self.name}/{diagram_name}.png', data_name = diagram_name)
            
        
        return
    
            
    # Construct data shown in document
    def construct(self, diagrams_per_page):
        
        counter = 0
        pages_data = []
        temp = []
        # Get all plots
        files = os.listdir(self.name)
        # Iterate over all created visualization
        for fname in files:
            # We want 3 per page
            if counter == 3:
                pages_data.append(temp)
                temp = []
                counter = 0

            temp.append(f'{self.name}/{fname}')
            counter += 1

        return [*pages_data, temp]
    
    def create_pdf(self):
        
        plots_per_page = self.construct(3)
        
        pdf = PDF()

        for elem in plots_per_page:
            pdf.print_page(elem)

        pdf.output('CentralityRepot.pdf', 'F')

In [4]:
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297
        
    def header(self):
        # Custom logo and positioning
        # Create an `assets` folder and put any wide and short image inside
        # Name the image `logo.png`
        '''
        self.image('assets/logo.png', 10, 8, 33)
        self.set_font('Arial', 'B', 11)
        self.cell(self.WIDTH - 80)
        self.cell(60, 1, 'Influence report', 0, 0, 'R')
        self.ln(20)
        '''
        return
        
    def footer(self):
        # Page numbers in the footer
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def page_body(self, images):
        # Determine how many plots there are per page and set positions
        # and margins accordingly
        if len(images) == 3:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
            self.image(images[2], 15, self.WIDTH / 2 + 90, self.WIDTH - 30)
        elif len(images) == 2:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
        else:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            
    def print_page(self, images):
        # Generates the report
        self.add_page()
        self.page_body(images)